In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import gc
import time
import os
import sys
#sys.path.append(os.path.join(os.path.dirname(__file__), '../models/'))
sys.path.append('../models')
from data_frame import DataFrame

config = tf.ConfigProto(allow_soft_placement = True)
config.gpu_options.allow_growth=True

In [ ]:
if not 3:
    print (1)

# Data Reading

In [ ]:
class DataReader(object):
    '''for reading data'''
    
    def __init__(self, data_dir):
        data_cols = [
            'item_id',
            'word_id',
            'history_length',
            'label'
        ]
        #-----------------
        # loading data
        #-----------------
        if TRACE_CODE == True:
            data = [np.load(os.path.join(data_dir, '{}_0.npy'.format(i)), mmap_mode='r') for i in data_cols]
        else:
            data = [np.load(os.path.join(data_dir, '{}.npy'.format(i)), mmap_mode='r') for i in data_cols]
        #
        self.test_df = DataFrame(columns=data_cols, data=data)

        print ('shape of whole data : {}'.format(self.test_df.shapes()))
        print ('loaded data')

        self.train_df, self.val_df = self.test_df.train_test_split(train_size=0.9, random_state = int(time.time()))

        print ('number of training example: {}'.format(len(self.train_df)))
        print ('number of validating example: {}'.format(len(self.val_df)))
        print ('number of testing example: {}'.format(len(self.test_df)))
        
    def train_batch_generator(self, batch_size, shuffle = True):
        return self.batch_generator(
            batch_size=batch_size,
            df=self.train_df,
            shuffle=shuffle,
            num_epochs=10000,
            is_test=False
        )

    def val_batch_generator(self, batch_size, shuffle = True):
        return self.batch_generator(
            batch_size=batch_size,
            df=self.val_df,
            shuffle=shuffle,
            num_epochs=10000,
            is_test=False
        )

    def test_batch_generator(self, batch_size,shuffle = False):
        return self.batch_generator(
            batch_size=batch_size,
            df=self.test_df,
            shuffle=shuffle,
            num_epochs=1,
            is_test=True
        )
    def batch_generator(self, batch_size, df, shuffle=True, num_epochs=10000, is_test=False):
        '''
        df: customized DataFrame object,
        '''
        # call our customized DataFrame object method batch_generator
        batch_gen = df.batch_generator(batch_size, shuffle = shuffle, num_epochs=num_epochs, allow_smaller_final_batch=is_test)
        # batch_gen is a generator
        for batch in batch_gen:
            # what batch_gen yield is also a customized Dataframe object.
            if not is_test:
                pass
            yield batch


In [ ]:
TRACE_CODE = False
reader = DataReader(data_dir ='../models/data/')


In [ ]:
word_id = np.load(os.path.join('../models/data/', '{}.npy'.format('word_id')))
history_length = np.load(os.path.join('../models/data/', '{}.npy'.format('history_length')))
label = np.load(os.path.join('../models/data/', '{}.npy'.format('label')))
item_id = np.load(os.path.join('../models/data/','{}.npy'.format('item_id')))
eval_set = np.load(os.path.join('../models/data/','{}.npy'.format('eval_set')))

In [ ]:
print (label.shape)
print (word_id.shape)
print (history_length.shape)
print (item_id.shape)
print (eval_set.shape)

In [ ]:
s = word_id[0]
s

In [ ]:
s1 = label[0]
s1

In [ ]:
s2 = history_length[0]
s2

In [ ]:
eval_set[0]

In [ ]:
df = pd.read_csv('../data/processed/mobile_training_w_word_id.csv')
df[df.item_id == 2]

# tf_utils

In [ ]:
import pprint as pp

def shape_of_tensor(tensor, dim=None):
    """
    Get tensor shape/dimension as list/int
    ======
    Args:
        tensor: tensor in tensorflow
        dim: int, the dimension of this tensor
    """
    if dim is None:
        # return list
        return tensor.shape.as_list()
    else:
        # t
        return tensor.shape.as_list()[dim]
    
def temporal_convolution_layer(inputs, output_units, convolution_width = 3, dilated = False,
                               causal=False, dilation_rate=[1], bias=True, activation=None, 
                               dropout=None, scope='temporal-convolution-layer', reuse=False):
    """
    Convolution over the temporal axis of sequence data.

    Args:
        inputs: Tensor of shape [batch size, max sequence length, input_units].
        output_units: Output channels for convolution.
        convolution_width: Number of timesteps to use in convolution.
        causal: Output at timestep t is a function of inputs at or before timestep t.
        dilated: Simple CNN or Dilated CNN
        dilation_rate:  Dilation rate along temporal axis.
        scope: str.
        reuse: boolean. If we would like to sharing this weight.

    Returns:
        Tensor of shape [batch size, max sequence length, output_units].

    """
    with tf.variable_scope(scope, reuse=reuse):
         if dilated == True:
            #-------------------
            # dilated CNN
            #-------------------
            if causal:
                # padding zero in the left side of sequence with (k-1) *d
                shift = int((convolution_width / 2) + (int(dilation_rate[0] - 1) / 2)) # modified to int
                pad = tf.zeros([tf.shape(inputs)[0], shift, inputs.shape.as_list()[2]])
                inputs = tf.concat([pad, inputs], axis=1)
            
            # filter weight
            W = tf.get_variable(
                name='weights',
                initializer=tf.contrib.layers.variance_scaling_initializer(),
                shape=[convolution_width, shape(inputs, 2), output_units]
            )
            # convolution = matrix multiplication + element-wise addition
            z = tf.nn.convolution(inputs, W, padding='SAME', dilation_rate=dilation_rate)
            # adding bias if True
            if bias:
                b = tf.get_variable(
                    name='biases',
                    initializer=tf.constant_initializer(),
                    shape=[output_units]
                )
                z = z + b
            # adding non-linear output if True
            z = activation(z) if activation else z
            # adding dropout if True
            z = tf.nn.dropout(z, dropout) if dropout is not None else z
            # output tensor of this hidden layer
            z = z[:, :-shift, :] if causal else z
        else:
            #-------------------
            # simple CNN
            #-------------------
            
            # filter weight
            W = tf.get_variable(
                name='weights',
                initializer=tf.contrib.layers.variance_scaling_initializer(),
                # shape = spatial_filter_shape + [in_channels, out_channels]
                shape=[convolution_width, shape_of_tensor(inputs, 2), output_units]
            )
            # convolution = matrix multiplication + element-wise addition
            z = tf.nn.convolution(input = inputs, 
                          filter = W, 
                          padding='SAME', 
                          )
            if bias:
                b = tf.get_variable(
                    name='biases',
                    initializer=tf.constant_initializer(),
                    shape=[output_units]
                )
                z = z + b

            # adding non-linear output if True
            z = activation(z) if activation else z
            # adding dropout if True
            z = tf.nn.dropout(z, dropout) if dropout is not None else z
            # output tensor of this hidden layer
            z = z[:, :-shift, :] if causal else z
            
        return z
    
    
def time_distributed_dense_layer(inputs, output_units, bias=True, activation=None, batch_norm=None,
                                 dropout=None, scope='time-distributed-dense-layer', reuse=False):
    """
    Applies a shared dense layer to each timestep of a tensor of shape [batch_size, max_seq_len, input_units]
    to produce a tensor of shape [batch_size, max_seq_len, output_units].

    output_units aka residual_channels: Number of channels to use for residual connections.

    Args:
        inputs: Tensor of shape [batch size, max sequence length, ...].
        output_units: Number of output units.
        activation: activation function.
        dropout: dropout keep prob.

    Returns:
        Tensor of shape [batch size, max sequence length, output_units].

    """
    with tf.variable_scope(scope, reuse=reuse):
        W = tf.get_variable(
            name='weights',
            initializer=tf.contrib.layers.variance_scaling_initializer(),
            shape=[shape_of_tensor(inputs, -1), output_units]
        )
        # matrix multiplication
        z = tf.einsum('ijk,kl->ijl', inputs, W)
        if bias:
            b = tf.get_variable(
                name='biases',
                initializer=tf.constant_initializer(),
                shape=[output_units]
            )
            z = z + b
        # doing batch_norm before activation is better for training nn.
        if batch_norm is not None:
            z = tf.layers.batch_normalization(z, training=batch_norm, reuse=reuse)
        #--------
        # In practice, activation fisst and then dropout
        #----------
        # adding non-linear output if True
        z = activation(z) if activation else z
        # adding dropout if True
        z = tf.nn.dropout(z, dropout) if dropout is not None else z
        return z

def sequence_softmax_loss(y, y_hat, sequence_lengths, max_sequence_length):
    """
    Calculates average softmax cross entropy on variable length sequences.

    Args:
        y: Label tensor of shape [batch size, max_sequence_length], which should be index of label.
        y_hat: Prediction tensor, [batch size, max_sequence_length, num_class], which should be unscaled score.
        sequence_lengths: Sequence lengths.  Tensor of shape [batch_size].
        max_sequence_length: maximum length of padded sequence tensor.

    Returns:
        batch_softmax_loss. 0-dimensional tensor.
        
    Reference:
        -Official: https://www.tensorflow.org/api_docs/python/tf/nn/sparse_softmax_cross_entropy_with_logits
        -Blog : https://blog.csdn.net/yc461515457/article/details/77861695
    """
    # softmax cross-entropy between y(y_true) and y_hat(y_pred)
    softmax_losses = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits= y_hat) # (?, max_sequence_length)
    # returns a boolean mask tensor for the first N positions of each cell.
    sequence_mask = tf.sequence_mask(lengths = sequence_lengths, maxlen=max_sequence_length) # (?, max_sequence_length)
    # convert boolean into 1 or 0
    sequence_mask = tf.cast(sequence_mask, tf.float32) # (?, max_sequence_length)
    # compute sum of loss over each timestep / number of real training example of this batch (aka batch loss)
    batch_softmax_loss = tf.reduce_sum(softmax_losses*sequence_mask) / tf.cast(tf.reduce_sum(sequence_lengths), tf.float32)
    return batch_softmax_loss



# TFBaseModel

In [ ]:
def get_optimizer(learning_rate, optimizer='adam'):
    '''
    It's for choosing optimizer given learning rate.
    '''
    if optimizer == 'adam':
        return tf.train.AdamOptimizer(learning_rate)
    elif optimizer == 'adagrad':
        return tf.train.AdagradOptimizer(learning_rate)
    elif optimizer == 'sgd':
        return tf.train.GradientDescentOptimizer(learning_rate)
    elif soptimizer == 'rms':
        return tf.train.RMSPropOptimizer(learning_rate, decay=0.95, momentum=0.9)
    else:
        # assert is a good way to tell other how to use this function for bug happening.
        #-------
        # standard way to pring the error
        #-------
        assert False, 'optimizer must be adam, adagrad, sgd, or rms'

def update_parameters(loss, optimizer = 'adam'):
    '''
    It's for optimizing and logging training parameters
    
    1.using gradient clipping to avoid gradient explosion and vanishment.
    
    Gradient clipping is most common in recurrent neural networks. 
    When gradients are being propagated back in time, they can vanish 
    because they they are continuously multiplied by numbers less than one.
    This is called the vanishing gradient problem. 
    This is solved by LSTMs and GRUs, and if you’re using a deep feedforward network, 
    This is solved by residual connections. 
    On the other hand, you can have exploding gradients too. 
    This is when they get exponentially large from being multiplied by numbers larger 
    than 1. Gradient clipping will clip the gradients between two numbers to prevent them from getting too large.

    '''
    #---------------
    # setting
    #---------------
    grad_clip = 5 # Clip gradients elementwise to have norm at most equal to grad_clip.
    regularization_constant = 0.1 # Regularization constant applied to all trainable parameters.
    enable_parameter_averaging = False # If true, model saves exponential weighted averages of parameters to separate checkpoint file.
    global_step = tf.Variable(0, trainable = False) # Optional Variable to increment by one after the variables have been updated.
    learning_rate_var = tf.Variable(0.0, trainable = False)
    
    #----------------
    # for understanding regularization
    #----------------
    trainable_variables_1 = tf.trainable_variables()[0]
    square_1 = tf.square(trainable_variables_1)
    sum_1 = tf.reduce_sum(square_1)
    sqrt = tf.sqrt(sum_1)
    #-----------------
    # we can customized our regularization on the parameters we like
    #-----------------
    if regularization_constant != 0:
        # l2_norm: is a 0-D tensor. 
        # we do l2-norm on each trainable's parameters.
        l2_norm = tf.reduce_sum([tf.sqrt(tf.reduce_sum(tf.square(param))) for param in tf.trainable_variables()]) # Returns list including all variables created with trainable=True
        # the smaller the loss is, the better do finish overfitting 
        loss = loss + regularization_constant*l2_norm
    #-----------------
    # optimizing
    #-----------------
    # define the optimizer
    optimizer = get_optimizer(learning_rate_var, optimizer=optimizer)
    # compute grads: return A list of (gradient, variable) pairs. Variable is always present, but gradient can be None.
    grads = optimizer.compute_gradients(loss)
    # standard way to do gradient clipping
    clipped = [(tf.clip_by_value(g, -grad_clip, grad_clip), v_) for g, v_ in grads]
    step = optimizer.apply_gradients(clipped, global_step = global_step)
    print ('step - whtat optimizer.apply_gradients returns', step)
    #-----------------
    # if using moving average techniques
    #-----------------
    if enable_parameter_averaging:
        ema = tf.train.ExponentialMovingAverage(decay=0.995)
        maintain_averages_op = ema.apply(tf.trainable_variables())
        with tf.control_dependencies([step]):
            step = tf.group(maintain_averages_op)
    else:
        step = step
    #--------------
    # logging
    #--------------
    logging.info('all parameters:')
    logging.info(pp.pformat([(var.name, shape_of_tensor(var)) for var in tf.global_variables()]))

    logging.info('trainable parameters:')
    logging.info(pp.pformat([(var.name, shape_of_tensor(var)) for var in tf.trainable_variables()]))

    logging.info('trainable parameter count:')
    logging.info(str(np.sum(np.prod(shape_of_tensor(var)) for var in tf.trainable_variables())))
    return grads, clipped , step, global_step, learning_rate_var

def save(step, averaged=False):
    '''
    save model
    
    Args:
        step: number checkpoint filenames by passing a value of step
        averaged:
    '''
    global saver
    global checkpoint_dir
    #---------
    # determine using which saver object
    #---------
    saver = saver_averaged if averaged else saver
    checkpoint_dir = checkpoint_dir_averaged if averaged else checkpoint_dir
    #---------
    #checkpoint_dir
    #---------
    if not os.path.isdir(checkpoint_dir):
        logging.info('creating checkpoint directory {}'.format(checkpoint_dir))
        os.mkdir(checkpoint_dir)

    model_path = os.path.join(checkpoint_dir, 'model')
    print ('========model_path in save========', model_path)
    logging.info('saving model to {}'.format(model_path))
    #saving
    saver.save(sess, model_path, global_step=step)
    
def restore(step=None, averaged=False):
    '''
    For using warm start technique
    '''
    global saver
    # checkpoint_dir: Directory where checkpoints are saved
    checkpoint_dir = 'checkpoints'

    saver = saver_averaged if averaged else saver
    checkpoint_dir = checkpoint_dir_averaged if averaged else checkpoint_dir
    if not step: 
        model_path = tf.train.latest_checkpoint(checkpoint_dir)
        print ('========model_path in restore========', model_path)
        logging.info('restoring model parameters from {}'.format(model_path))
        saver.restore(sess, model_path)
    else:
        model_path = os.path.join(
            checkpoint_dir, 'model{}-{}'.format('_avg' if averaged else '', step)
        )

        logging.info('restoring model from {}'.format(model_path))
        saver.restore(sess, model_path)


In [ ]:
'''
Case1:Softmax
1.輸出3維度的藍色點點
3.Softmax


Case2:CRF
1.輸出3為度的藍色點點
2.CRF

Prediction
取最大維度的值

'''

In [ ]:
from data_utils import get_glove_vectors
from data_utils import load_vocab_and_return_word_to_id_dict
sys.path.append('/home/ld-sgdev/yunrui_li/ner_project/brand_recognition_bio_FE/py_model')
from utils import init_logging
import logging
log_dir = 'log/' # log path
init_logging(log_dir)

In [ ]:
dim_word = 300
max_seq_length = 122
num_layers = 2
hidden_size_cnn = 100
k = 3
ntags = 3 # n_class
USE_PRETRAINED = True
trainable_embedding = False
filename_words_vec = "../models/data/wordvec/word2vec.npz".format(dim_word)
filename_words_voc = "../models/data/wordvec/words_vocab.txt"

nwords = len(load_vocab_and_return_word_to_id_dict(filename_words_voc))
embeddings = (get_glove_vectors(filename_words_vec) if USE_PRETRAINED else None)
embeddings = embeddings.astype(np.float32)
enable_parameter_averaging = False
gc.collect()
with tf.Graph().as_default() as g:
    ####################################
    # Step1: get input_sequences 
    ####################################

    #------------
    # 1-D  
    #------------
    item_id = tf.placeholder(tf.int32, [None])
    history_length = tf.placeholder(tf.int32, [None]) # It's for arg of lstm model: sequence_length, == len(is_ordered_history)
    #------------   
    # 2-D  
    #------------
    word_id = tf.placeholder(tf.int32, [None, max_seq_length]) 
    label = tf.placeholder(tf.int32, [None, max_seq_length]) # [batch_size, num_class]

    #------------
    # boolean parameter
    #------------
    keep_prob = tf.placeholder(tf.float32)
    is_training = tf.placeholder(tf.bool)
    
    #------------
    # word_embedding: get char embeddings matrix
    #------------
    if embeddings is None:
        logging.info('WARNING: randomly initializing word vectors')
        word_embeddings = tf.get_variable(
        shape = [nwords, dim_word],
        name = 'word_embeddings',
        dtype = tf.float32,
        )
    else:
        word_embeddings = tf.get_variable(
        initializer = embeddings, # it will hold the embedding
        #shape = [word2vec.shape[0], word2vec.shape[1]], # [num_vocabulary, embeddings_dim]
        trainable = trainable_embedding,
        name = 'word_embeddings',
        dtype = tf.float32
        )
    word_representation = tf.nn.embedding_lookup(params = word_embeddings, ids = word_id)
    x_word = tf.concat([
    word_representation,
    # tf_idf:for product_name
        ], axis=1) # (?, 122, 300)
    
    ####################################
    # Step2: calculate_outputs 
    ####################################
    
    #-------------------------
    # NN architecuture-Simple CNN
    #-------------------------
    print ('Original features : {}'.format(x_word.shape))
    for i in range(num_layers):
        if i == 0:
            conv = temporal_convolution_layer(x_word, 
                                       output_units = hidden_size_cnn,
                                       convolution_width = k,
                                       dilated = False,
                                       causal = False,
                                       bias=True,
                                       activation=None, 
                                       dropout=None,
                                       scope='cnn-{}'.format(i),
                                       reuse = False,
                                      )
        else:
            conv = temporal_convolution_layer(conv, 
                                       output_units = hidden_size_cnn,
                                       convolution_width = k,
                                       dilated = False,
                                       causal = False,
                                       bias=True,
                                       activation=None, 
                                       dropout=None,
                                       scope='cnn-{}'.format(i),
                                       reuse = False,
                                      )
            
        print ('CNN-{} layer : {}'.format(i, conv.shape))
    # output layer (linear)
    y_hat = time_distributed_dense_layer(conv, ntags, activation=None, scope='output-layer') # (?, 122, 3)
    print ('Output layer : {}'.format(y_hat.shape))
    print ('y_true : {}'.format(label.shape))
    #--------------
    # for second-level model
    #--------------
    prediction_tensors = {
        'item_id':item_id,
        'word_id':word_id,
        'final_states':conv, # 修改不要全部max_seq_lenghth都存, 只存到history_length的長度(save memory)
        'final_predictions':y_hat,
    }
    
    ####################################
    # Step3: calculate_loss + optimizing
    ####################################
    loss,softmax_losses,sequence_mask = sequence_softmax_loss(y = label, y_hat = y_hat, sequence_lengths = history_length, max_sequence_length = max_seq_length)
    grads, clipped , step, global_step, learning_rate_var  = update_parameters(loss)
    ####################################
    # Step4: saving the model 
    ####################################    
    # create saver object
    # max_to_keep: indicates the maximum number of recent checkpoint files to keep.
    saver = tf.train.Saver(max_to_keep = 1)
    if enable_parameter_averaging:
        saver_averaged = tf.train.Saver(ema.variables_to_restore(), max_to_keep=1)    

    #-------------------------
    # standard
    #-------------------------
    init = tf.global_variables_initializer()
    

In [ ]:
'''
-test time_distributed_dense_layer
-test sequence_softmax_loss
-understanding how to determine grad_clip==> empirical
-understanding save and restore and fit function, and make them start training
'''

In [ ]:
from collections import deque # for computing Train/validation losses are averaged over the last loss_averaging_window

warm_start_init_step = 0 # If nonzero, model will resume training a restored model beginning at warm_start_init_step.
batch_size = 128
loss_averaging_window = 10
num_validation_batches = 1
num_training_steps = 10
learning_rate=0.001
log_interval = 1
min_steps_to_checkpoint =5
early_stopping_steps = 10


base_dir = './'
checkpoint_dir = os.path.join(base_dir, 'checkpoints')

with tf.Session(graph=g, config = config) as sess:
    ####################################
    # 1. fit
    ####################################
    if warm_start_init_step:
        # continue the optimization at a recent checkpoint instead of having to restart the optimization from the beginning
        restore(warm_start_init_step)
        step = warm_start_init_step
    else:
        # start the optimization from the beginning
        sess.run(init) # Run the initializer
        step = 0
        
    sess.run(init)
    #--------------
    # create generator for batch training
    #--------------
    train_generator = reader.train_batch_generator(batch_size) # it will yield our customized dataframe
    val_generator = reader.val_batch_generator(num_validation_batches*batch_size)
    
    # create deque object (list-like): for only considering last 100 training steps
    train_loss_history = deque(maxlen = loss_averaging_window)
    val_loss_history = deque(maxlen= loss_averaging_window)

    best_validation_loss, best_validation_tstep = float('inf'), 0
    restarts = 0
    while step < num_training_steps:
        #-----------------------
        # validating 
        #-----------------------
        val_batch_df = next(val_generator) # it's also a generator.

        val_feed_dict = {}

            
        for placeholder_name, data in val_batch_df:
            if placeholder_name is not None:
                if placeholder_name in vars():
                    val_feed_dict.update({globals()[placeholder_name]: data})
        val_feed_dict.update({learning_rate_var: learning_rate})

    
        if keep_prob is not None:
            val_feed_dict.update({keep_prob: 1.0}) # 在validating 和testing時, keep_prob:1.0

        if is_training is not None:
            val_feed_dict.update({is_training: False})
        # feeding data into the graph we designed
        [val_loss] = sess.run(
            fetches=[loss],
            feed_dict=val_feed_dict
        )
        print ('val_loss', val_loss)
        val_loss_history.append(val_loss)
     
        #-----------------------
        # training
        #-----------------------
        train_batch_df = next(train_generator)
        train_feed_dict = {}
        
        for placeholder_name, data in train_batch_df:
            if placeholder_name is not None:
                #print ('placeholder_name === train', placeholder_name)
                if placeholder_name in vars():
                    #print ('placeholder_name === train',globals()[placeholder_name])
                    train_feed_dict.update({globals()[placeholder_name]: data})

        train_feed_dict.update({learning_rate_var: learning_rate})
        if keep_prob is not None:
            train_feed_dict.update({keep_prob: 1.0})
        if is_training is not None:
            train_feed_dict.update({is_training: True})

        [train_loss] = sess.run(
            fetches=[loss],
            feed_dict=train_feed_dict
        )
        print ('train_loss', train_loss)
        train_loss_history.append(train_loss)
    
        #----------
        # logging: log the training and validating loss every log_interval training steps
        #----------
        if step % log_interval == 0:
            # compute average batch training loss over the last loss_averaging_window steps
            avg_train_loss = sum(train_loss_history) / len(train_loss_history)
            print ('avg_train_loss', avg_train_loss)
            avg_val_loss = sum(val_loss_history) / len(val_loss_history)
            print ('avg_val_loss', avg_val_loss)
            metric_log = (
                "[[step {:>8}]]     "
                "[[train]]     loss: {:<12}     "
                "[[val]]     loss: {:<12}     "
            ).format(step, round(avg_train_loss, 8), round(avg_val_loss, 8))
            logging.info(metric_log)

            #------------------
            # early stopping
            #------------------
            if avg_val_loss < best_validation_loss:
                # update the best_validation_loss
                best_validation_loss = avg_val_loss
                best_validation_tstep = step # best_validation_tstep: for recording the model_path in restore function
                if step > min_steps_to_checkpoint:
                    # saving 
                    save(step)
                    if enable_parameter_averaging:
                        save(step, averaged=True)

            if step - best_validation_tstep > early_stopping_steps:
                #----------------
                # handling loss plateaus:  halving earning rate will be repeated more times. 
                #----------------
                if num_restarts is None or restarts >=  num_restarts:
                    # stop training
                    logging.info('best validation loss of {} at training step {}'.format(
                        best_validation_loss, best_validation_tstep))
                    logging.info('early stopping - ending training.')

                if restarts < num_restarts:
                    # 1.the best checkpoint will be restored.
                    # 2.keep traning but halving the learning rate and early_stopping_steps
                    restore(best_validation_tstep)
                    logging.info('halving learning rate')
                    learning_rate /= 2.0
                    early_stopping_steps /= 2
                    step = best_validation_tstep # start from the best_validation_tstep
                    restarts += 1
        # for terminating while
        step += 1
        print ('**********step**********', step)
    #----------------
    # for the case: num_training_steps < = min_steps_to_checkpoint(Basically, it won't happen)
    #----------------
    if step <= min_steps_to_checkpoint:
        best_validation_tstep = step
        save(step)
        if enable_parameter_averaging:
            save(step, averaged=True)

    logging.info('num_training_steps reached - ending training')

    ####################################
    # 2.restore
    ####################################
    restore()
    ####################################
    # 3.predict
    ####################################
    
    chunk_size = 3
    prediction_dir = 'predictions'
    if not os.path.isdir(prediction_dir):
        os.makedirs(prediction_dir)
    
    #--------------------
    # for input of 2nd-level
    #--------------------
    prediction_dict = {}
    for tensor_name, value in prediction_tensors.items():
        prediction_dict.update({tensor_name: []})
        
    test_generator = reader.test_batch_generator(chunk_size)
    for i, test_batch_df in enumerate(test_generator):
        if i % 100 == 0:
            print (i*chunk_size)

        #--------------
        # preparing feed_dict
        #--------------
        test_feed_dict = {}
        for placeholder_name, data in test_batch_df:
            if placeholder_name is not None:
                #print ('placeholder_name === val', placeholder_name)
                if placeholder_name in vars():
                    #print ('placeholder_name === val',globals()[placeholder_name])
                    test_feed_dict.update({globals()[placeholder_name]: data})
        test_feed_dict.update({learning_rate_var: learning_rate})

        if keep_prob in vars():
            # Pracitcally, After finishing the training, it is important to turn off the dropout during development and testing.
            # Otherwise, the prediction of this model is not stable since dropout add uncertainties to it.
            test_feed_dict.update({keep_prob: 1.0})
        if is_training in vars():
            test_feed_dict.update({is_training: False})
        print ('test_feed_dict',test_feed_dict)
        #-----------
        # feeding the data into session
        #-----------
        tensor_names, tf_tensors = zip(*prediction_tensors.items()) 
        np_tensors = sess.run(
            fetches = list(tf_tensors),
            feed_dict=test_feed_dict
        )  # return list
        
        for tensor_name, tensor in zip(list(tensor_names), np_tensors):
            print ('=====tensor_name in test =====', tensor_name)
            print ('=====tensor in test =====', tensor.shape, tensor)
            prediction_dict[tensor_name].append(tensor)
    #-------------
    # save the prediction result
    #-------------
    for tensor_name, tensor in prediction_dict.items():
        # tensor: list of array w shape of (batch_size, )
        # tensor_name: str
        # 
        np_tensor = np.concatenate(tensor, axis = 0)
        save_file = os.path.join(prediction_dir, '{}.npy'.format(tensor_name))
        print ('saving {} with shape {} to {}'.format(tensor_name, np_tensor.shape, save_file))
        logging.info('saving {} with shape {} to {}'.format(tensor_name, np_tensor.shape, save_file))
        np.save(save_file, np_tensor)    
    


In [ ]:
# early stopping based on f1-scr

In [ ]:
# 1.design a customized plot funtion to observe training/validation loss versus epoch
# 2.build tensorflow base model
# 3.suvey about how to tune deep learning model using Bayesian Optimization
### reference: https://github.com/Hvass-Labs/TensorFlow-Tutorials/blob/master/19_Hyper-Parameters.ipynb

In [ ]:
# 可以透過is_training 傳入variable決定這個batch要怎麼運算

In [ ]:
if not 4:
    print (4)

In [2]:
d = set() & set()

In [3]:
len(d)

0